In [1]:
pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: keras
    Found existing installation: keras 3.0.5
    Uninstalling keras-3.0.5:
      Successfully uninstalled keras-3.0.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
Note: you may need to restart the kernel to use updated packages.


In [2]:

from keras import preprocessing
import os
import numpy as np
import pandas as pd
import keras
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.utils import to_categorical
from scipy import ndimage
from keras import models
from keras import layers
from keras.models import load_model
from keras.models import save_model
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout
import tensorflow as tf
from keras.preprocessing import image
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

2024-03-12 04:25:46.366698: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-12 04:25:46.366824: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-12 04:25:46.536170: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# Function to preprocess the dataset
def preprocess_dataset(data):
    data_resized = cv2.resize(data, (224, 224))    # resize
    data_normalized = data_resized / 255.0         # normalize
    return data_normalized

# Function to preprocess the test dataset
def preprocess_test_dataset(test_dataset_path):
    preprocessed_data = np.empty((0, 224, 224, 3), dtype=np.float32)  # To store preprocessed images
    labels = []              # To store labels

    test_images_paths = [os.path.join(test_dataset_path, filename) for filename in os.listdir(test_dataset_path) if
                         filename.endswith(('.jpg', '.jpeg', '.png'))]   # Get paths of all images in the test dataset directory

    for image_path in test_images_paths:
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Unable to read the image: {image_path}")
            continue

        # Preprocess the image
        preprocessed_image = preprocess_dataset(image)
        preprocessed_data = np.concatenate((preprocessed_data, preprocessed_image[None]), axis=0)

        # Extract folder name as the label
        label = os.path.basename(os.path.dirname(image_path))
        labels.append(label)

    # Convert the preprocessed data to a numpy array
    preprocessed_data = np.array(preprocessed_data)

    return preprocessed_data, labels

# the train and validation datasets
path = "/kaggle/input/products2/dataset/train"

def Train_data_preprocessing_(path):
    image_generator = ImageDataGenerator(
        rescale=1/255,
        rotation_range=10,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2
    )

    train_dataset = image_generator.flow_from_directory(
        batch_size=32,
        directory=path,
        shuffle=True,
        target_size=(224, 224),
        subset="training",
        class_mode='categorical'
    )

    test_dataset = image_generator.flow_from_directory(
        batch_size=32,
        directory=path,
        shuffle=True,
        target_size=(224, 224),
        subset="validation",
        class_mode='categorical'
    )

    return train_dataset, test_dataset

train, test = Train_data_preprocessing_(path)

Found 37 images belonging to 5 classes.
Found 7 images belonging to 5 classes.


In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train,
    epochs=10,
    validation_data=test
)

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model2.h5', save_best_only=True)

# Training
model.fit(train, epochs=50, validation_data=test, callbacks=[early_stopping, model_checkpoint], verbose=1)

model.save('model3.h5')

Epoch 1/10


I0000 00:00:1710217713.773057     138 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2/2 [==============================] - 9s 1s/step - loss: 1.7960 - accuracy: 0.2162 - val_loss: 5.7735 - val_accuracy: 0.2857
Epoch 2/10
2/2 [==============================] - 1s 750ms/step - loss: 3.6356 - accuracy: 0.3243 - val_loss: 1.9204 - val_accuracy: 0.2857
Epoch 3/10
2/2 [==============================] - 1s 236ms/step - loss: 1.9259 - accuracy: 0.1892 - val_loss: 1.6178 - val_accuracy: 0.1429
Epoch 4/10
2/2 [==============================] - 1s 756ms/step - loss: 1.6091 - accuracy: 0.1892 - val_loss: 1.6096 - val_accuracy: 0.1429
Epoch 5/10
2/2 [==============================] - 1s 751ms/step - loss: 1.6066 - accuracy: 0.2432 - val_loss: 1.5795 - val_accuracy: 0.2857
Epoch 6/10
2/2 [==============================] - 1s 203ms/step - loss: 1.5690 - accuracy: 0.2973 - val_loss: 1.5252 - val_accuracy: 0.2857
Epoch 7/10
2/2 [==============================] - 1s 196ms/step - loss: 1.4935 - accuracy: 0.3243 - val_loss: 1.4865 - val_accuracy: 0.5714
Epoch 8/10
2/2 [==================

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2/2 [==============================] - 2s 1s/step - loss: 1.3051 - accuracy: 0.4595 - val_loss: 1.3159 - val_accuracy: 0.4286
Epoch 2/50
2/2 [==============================] - 1s 200ms/step - loss: 1.2478 - accuracy: 0.4595 - val_loss: 1.3234 - val_accuracy: 0.2857
Epoch 3/50
2/2 [==============================] - 2s 1s/step - loss: 1.2299 - accuracy: 0.4595 - val_loss: 1.2392 - val_accuracy: 0.5714
Epoch 4/50
2/2 [==============================] - 1s 773ms/step - loss: 1.1804 - accuracy: 0.5135 - val_loss: 1.2624 - val_accuracy: 0.4286
Epoch 5/50
2/2 [==============================] - 2s 2s/step - loss: 1.1679 - accuracy: 0.5135 - val_loss: 1.1862 - val_accuracy: 0.4286
Epoch 6/50
2/2 [==============================] - 2s 1s/step - loss: 1.1016 - accuracy: 0.4595 - val_loss: 1.1789 - val_accuracy: 0.2857
Epoch 7/50
2/2 [==============================] - 1s 201ms/step - loss: 1.1143 - accuracy: 0.4865 - val_loss: 1.2482 - val_accuracy: 0.5714
Epoch 8/50
2/2 [===========================

In [8]:
# Path to the test dataset
test_dataset_path = "/kaggle/input/products2/dataset/test"

def test_images(test_dataset_path, model):
    preprocessed_data, labels = preprocess_test_dataset(test_dataset_path)

    if preprocessed_data.shape[0] == 0:
        print("No valid images in the test dataset.")
        return

    # Make predictions
    predictions = model.predict(preprocessed_data)
    predicted_labels = np.argmax(predictions, axis=1)

    results = pd.DataFrame({'image_id': labels, 'predicted_folder': predicted_labels})
    print(results)

    #csv_file_path = 'predicted_folders333.csv'
    #results.to_csv(csv_file_path, index=False)

# Load the model
model2 = load_model('model3.h5')

# Set the path to the test dataset
test_dataset_path = "/kaggle/input/products2/dataset/test"

# Call the test_images function
test_images(test_dataset_path, model2)

1/1 [==============================] - 0s 82ms/step
   image_id  predicted_folder
0      test                 2
1      test                 4
2      test                 1
3      test                 1
4      test                 1
5      test                 1
6      test                 4
7      test                 1
8      test                 4
9      test                 1
10     test                 1
11     test                 2
12     test                 1
13     test                 2
